In [1]:
import numpy as np
import os
import networkx as nx
import random as rand
#from random import sample
#from random import random
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics.pairwise import cosine_similarity
from numpy import dot
from numpy.linalg import norm


from collections import Counter
import matplotlib.pyplot as plt

In [ ]:
all_data = []
all_edges = []

with open("cora.content",'r') as f:
  all_data.extend(f.read().splitlines())
with open("cora.cites",'r') as f:
  all_edges.extend(f.read().splitlines())



In [3]:
categories =  ['Reinforcement_Learning', 'Theory', 'Case_Based', 'Genetic_Algorithms', 'Probabilistic_Methods', 'Neural_Networks', 'Rule_Learning']
sorted(categories)
label_encoder = {}
i = 0
for cat in sorted(categories):
  label_encoder[cat] = i
  i +=1
label_encoder

{'Case_Based': 0,
 'Genetic_Algorithms': 1,
 'Neural_Networks': 2,
 'Probabilistic_Methods': 3,
 'Reinforcement_Learning': 4,
 'Rule_Learning': 5,
 'Theory': 6}

In [4]:
#parse the data
labels = []
nodes = []
X = []
element_to_ind  = {}

for i,data in enumerate(all_data):
    elements = data.split('\t')
    labels.append(label_encoder[elements[-1]])
    X.append(elements[1:-1])
    nodes.append(elements[0])
    element_to_ind[elements[0]]= i
X = np.array(X,dtype=int)
N = X.shape[0] #the number of nodes
F = X.shape[1] #the size of node features
print('X shape: ', X.shape)


#parse the edge
edge_list=[]
for edge in all_edges:
    e = edge.split('\t')
    edge_list.append((e[0],e[1]))

print('\nNumber of nodes (N): ', N)
print('\nNumber of features (F) of each node: ', F)
print('\nCategories: ', set(labels))

num_classes = len(set(labels))
print('\nNumber of classes: ', num_classes)

X shape:  (2708, 1433)

Number of nodes (N):  2708

Number of features (F) of each node:  1433

Categories:  {0, 1, 2, 3, 4, 5, 6}

Number of classes:  7


In [5]:
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edge_list)
G = nx.relabel_nodes(G, element_to_ind)
print('Graph info: ', G)

Graph info:  Graph with 2708 nodes and 5278 edges


In [6]:
nodes = list(G.nodes)
print(len(nodes))
list(G.neighbors(0))

2708


[258, 544, 8, 435, 14]

In [7]:
df = pd.DataFrame(list(zip(nodes, labels,X)),columns =['node', 'label','features'])
print(len(df))
df.head()

2708


,node,label,features
0,0,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,5,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,2,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,3,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,4,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [8]:
Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
G = G.subgraph(Gcc[0])
gcc_nodes = list(G.nodes)

In [9]:
df = df.loc[df['node'].isin(gcc_nodes)]
df['node'] = list(range(len(df))) #rename nodes
df.head()

<ipython-input-9-a01e0eb6d909>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['node'] = list(range(len(df))) #rename nodes


,node,label,features
0,0,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,5,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,2,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,3,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,4,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [10]:
train = df.groupby('label', group_keys=False).apply(lambda x: x.sample(20))
G = nx.relabel_nodes(G, df['node'])

In [11]:
def create_transition_matrix(g):
    vs = list(g.nodes)
    n = len(vs)
    adj = nx.adjacency_matrix(g)
    transition_matrix = adj/adj.sum(axis = 1)
    if sum(transition_matrix[0] != 1):
      return transition_matrix.transpose()
    else:
      return transition_matrix

In [12]:
def create_transition_matrix_same_label(g, label):
  G2 = nx.Graph()
  G2.add_nodes_from(G.nodes)
  for i in range(len(G.nodes)):
    if labels[i] == label:
      for j in range(i, len(G.nodes)):
        if labels[i] == labels[j]:
          from_node = nodes[i]
          #print(from_node)
          to_node = nodes[j]
          #print(to_node)
          G2.add_edge(from_node, to_node)
  adj = nx.adjacency_matrix(G2)
  transition_matrix = adj/adj.sum(axis=1)
  transition_matrix[np.isnan(transition_matrix)] = 0
  #transition_matrix = transition_matrix.toarray()
  return transition_matrix

In [13]:
all_teleport_matrices = []
for label in range(7):
  temp_p = [0] * 2485
  #G2 = nx.Graph()
  #G2.add_nodes_from(G.nodes)
  low = label*20
  high = (label+1)*20
  for i in range(140):
    if low <= i < high:
      temp_p[train['node'].iloc[i]] = 1/20
      #for j in range(i, 140):
        #if label*20 <= i < (label+1)*20:
          #from_node = train['node'].iloc[i]
          #to_node = train['node'].iloc[j]
          #print("from_node", from_node, " to_node ", to_node)
          #G2.add_edge(from_node, to_node)
          #print("edge")
        #if train['label'].iloc[i] == train['label'].iloc[j]:
          #print(labels[i])
          #print(train['label'].iloc[i])
          #print(train['label'].iloc[j])
          #print(labels[j])
          #from_node = train['node'].iloc[i]
          #print(from_node)
          #to_node = train['node'].iloc[j]
          #print(to_node)
          #G2.add_edge(from_node, to_node)
  #adj = nx.adjacency_matrix(G2)
  #print(adj)
  #transition_matrix = adj/adj.sum(axis=1)
  #print(transition_matrix)
  #transition_matrix[np.isnan(transition_matrix)] = 0
  #print(transition_matrix)
  #transition_matrix = transition_matrix.toarray()
  #all_teleport_matrices.append(transition_matrix)
  #print(transition_matrix.transpose())
  all_teleport_matrices.append(temp_p)
  print(label)

0
1
2
3
4
5
6


In [14]:
def random_walk(g, num_steps, start_node, transition_matrix = None):
  if transition_matrix is None:
    transition_matrix = create_transition_matrix(g)
  end_nodes = []
  v = start_node
  #print(transition_matrix)
  for i in range(num_steps) :
    #temp = []
    #v.astype(int)
    #for trow in transition_matrix:
      #print(trow[v])
      #temp.append(trow[v])
    PMF = transition_matrix[v,]#temp   #transition_matrix[v,]
    #print(sum(PMF))
    temp2 = np.random.choice(range(g.number_of_nodes()), 1, p = PMF)
    temp2.astype(int)
    #print(temp2[0])
    v=temp2[0]
    #end_nodes.append(v)
  #print(end_nodes)
  return v
  #perform a random walk
  #return v

In [21]:
seeds_dict = {predicted:list(train[train['label'] == predicted]['node']) for predicted in range(7)}

def random_walk_with_teleportation(g, num_steps, start_node, tp, predicted, transition_matrix, transition_matrix_no_teleport):
  #transition_matrix_no_teleport = create_transition_matrix(g)
  #transition_matrix_no_teleport = transition_matrix_no_teleport.toarray()
  #transition_matrix = transition_matrix.toarray()
  end_nodes = []
  v = start_node
  for i in range(num_steps) :
    if rand.random() >= tp :
      PMF = transition_matrix_no_teleport[v,]
      temp = np.random.choice(range(g.number_of_nodes()), 1, p = PMF)
      temp.astype(int)
      v=temp[0]
    else :
      PMF = transition_matrix
      temp = np.random.choice(range(g.number_of_nodes()), 1, p = PMF)
      temp.astype(int)
      v=temp[0]
    #end_nodes[[i]] = v
  return v
  #modify random walk code to add teleportation.
  #you can only teleport to a node belonging to the same class as the start node


  #return v


In [ ]:
#pagerank. NO teleportation, NO tfidf.
transition_matrix = create_transition_matrix(G)

num_samples = 1000
num_walk_steps = 100

visiting_freq_label = []
for i in range(transition_matrix.shape[0]):
  visiting_freq_label.append([0,0,0,0,0,0,0])

visiting_freq = [0 for i in range(transition_matrix.shape[0])]


for train_node,predicted in zip(train['node'],train['label']):
  #print (train_node,predicted)
  for i in range(num_samples):
      #print(i)
      start_point = train_node
      end_node = random_walk(G, num_walk_steps, start_point, transition_matrix)
      visiting_freq_label[end_node][predicted] += 1
      visiting_freq[end_node] +=1

In [ ]:
count = 0 #these many nodes remain unvisited.
for vf in visiting_freq:
  if vf ==0:
    count+=1
print('unvisited = ', count)
visiting_freq_label = np.asarray(visiting_freq_label)
preds = np.argmax(visiting_freq_label,axis = 1)
print(classification_report(df['label'], preds))
accuracy_score(df['label'], preds)

In [ ]:
#pagerank. WITH telportation, without tfidf

#repeat above expeiment but this time use the teleportation random walk
transition_matrix = create_transition_matrix(G)
#get metrics

num_samples = 1000
num_walk_steps = 100

visiting_freq_label = []
for i in range(transition_matrix.shape[0]):
  visiting_freq_label.append([0,0,0,0,0,0,0])

visiting_freq = [0 for i in range(transition_matrix.shape[0])]

label_visit = [0]*7

deletelater = 0

for train_node,predicted in zip(train['node'],train['label']):
  #print (train_node,predicted)
  #teleport_matrix = create_transition_matrix_same_label(G, labels[start_point])
  print(deletelater)
  deletelater += 1
  teleport_matrix = all_teleport_matrices[labels[train_node]]
  for i in range(num_samples):
      #print(i)
      start_point = train_node

      end_node = random_walk_with_teleportation(G, num_walk_steps, start_point, 0, predicted, teleport_matrix, transition_matrix)
      visiting_freq_label[end_node][predicted] += 1
      visiting_freq[end_node] +=1
      visit_index = labels[end_node]
      label_visit[visit_index] += 1

In [ ]:
count = 0 #these many nodes remain unvisited.
for vf in visiting_freq:
  if vf ==0:
    count+=1
print('unvisited = ', count)
visiting_freq_label = np.asarray(visiting_freq_label)
preds = np.argmax(visiting_freq_label,axis = 1)
print(classification_report(df['label'], preds))
accuracy_score(df['label'], preds)
print(label_visit)

In [ ]:
vs = list(G.nodes)
n = len(vs)
adj = nx.adjacency_matrix(G)
transition = np.zeros((len(G.nodes), len(G.nodes)))

temp = df['features']
for n1 in G.nodes:
  #print(n1)
  idx = df.index[df['node']==n1].values[0]
  n1_features = temp[idx]
  for n2 in G.nodes:
    if G.has_edge(n1, n2):
      idx2 = df.index[df['node']==n2].values[0]
      #print(idx2[0])
      n2_features = temp[idx2]
      #print(n2_features)
      #print(dot(n1_features, n2_features))
      #print(norm(n1_features)*norm(n2_features))
      cos_sim = dot(n1_features, n2_features)/(norm(n1_features)*norm(n2_features))
      #cos_sim = cosine_similarity(n1_features, n2_features)
      #print(cos_sim)
      transition[n1, n2] = np.exp(cos_sim)
transition = transition/transition.sum(axis=1)
if (transition.sum() != 1):
  transition = transition.transpose()
#for n1 in nodes:
  #for n2 in nodes:
    # if there is an edge between n1 and n2:
      # cos_sim = compute cosine similarity between features of n1 and n2
      # transition[n1,n2] = np.exp(cos_sim) #neumerator of softmax. #why do we need softmax?
#divide the values in transition by denominator of softmax. how will you do this?

In [ ]:
#pagerank. Without teleportation. WITH TFIDF
transition_matrix = transition

#perfrom pagerank using our tf_idf based transition matrix
#use randon walk without teleporation
#get metrics
num_samples = 1000
num_walk_steps = 100

visiting_freq_label = []
for i in range(transition_matrix.shape[0]):
  visiting_freq_label.append([0,0,0,0,0,0,0])

visiting_freq = [0 for i in range(transition_matrix.shape[0])]

label_visit = [0]*7

deletelater = 0
for train_node,predicted in zip(train['node'],train['label']):
  #print (train_node,predicted)
  print(deletelater)
  deletelater += 1
  for i in range(num_samples):
      #print(i)
      start_point = train_node
      end_node = random_walk(G, num_walk_steps, start_point, transition_matrix)
      visiting_freq_label[end_node][predicted] += 1
      visiting_freq[end_node] +=1
      label_visit[visit_index] += 1

In [ ]:
count = 0 #these many nodes remain unvisited.
for vf in visiting_freq:
  if vf ==0:
    count+=1
print('unvisited = ', count)
visiting_freq_label = np.asarray(visiting_freq_label)
preds = np.argmax(visiting_freq_label,axis = 1)
print(classification_report(df['label'], preds))
accuracy_score(df['label'], preds)
print(label_visit)

In [ ]:
#pagerank. WITH teleportation WITH TFIDF
transition_matrix = transition

#same as above, except use random walk with teleportation
#get metrics
num_samples = 1000
num_walk_steps = 100

visiting_freq_label = []
for i in range(transition_matrix.shape[0]):
  visiting_freq_label.append([0,0,0,0,0,0,0])

visiting_freq = [0 for i in range(transition_matrix.shape[0])]

#deletelater = 0

label_visit = [0]*7

for train_node,predicted in zip(train['node'],train['label']):
  #print (train_node,predicted)
  #teleport_matrix = create_transition_matrix_same_label(G, labels[start_point])
  #print(deletelater)
  deletelater += 1
  teleport_matrix = all_teleport_matrices[labels[train_node]]
  for i in range(num_samples):
      #print(i)
      start_point = train_node

      end_node = random_walk_with_teleportation(G, num_walk_steps, start_point, 0.2, predicted, teleport_matrix, transition_matrix)
      visiting_freq_label[end_node][predicted] += 1
      visiting_freq[end_node] +=1
      label_visit[visit_index] += 1

In [ ]:
count = 0 #these many nodes remain unvisited.
for vf in visiting_freq:
  if vf ==0:
    count+=1
print('unvisited = ', count)
visiting_freq_label = np.asarray(visiting_freq_label)
preds = np.argmax(visiting_freq_label,axis = 1)
print(classification_report(df['label'], preds))
accuracy_score(df['label'], preds)
print(label_visit)